<a href="https://colab.research.google.com/github/yuzukiiii16/ml-course/blob/main/%E3%80%90Ch4%E3%80%91K_means_Clustering%EF%BC%BF%E7%AC%ACO%E7%B5%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>說明：</h1>
<h3>你需要修改的地方會用底下的標記框出來，請依照註解的提示修改</h3>



```
# ==========================


# ==========================
```



# 匯入必要套件

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

#匯入初步處理好的資料


In [ ]:
FILE_NAME="New_York_Airbnb_Open_Data_2024_cleaned.csv"

!curl -L -o $FILE_NAME "https://drive.google.com/uc?export=download&id=1tMS63gsms0AEglD0Ac6WOrWEtaTvFsCX"

###資料下載後，檢查是否成功讀取，如果能看到資料代表成功

In [ ]:
df = pd.read_csv("New_York_Airbnb_Open_Data_2024_cleaned.csv", encoding="utf-8")
df.head()

# 選擇要分析的數值特徵

In [ ]:
# ==========================
# 請修改以下程式碼
# 說明: 選出要分析的數值特徵，放入numerical_features

numerical_features = []
# ==========================

#特徵工程

## 對數值特徵進行 Log 轉換
使用 np.log1p也就是 log(1+x)，因為可以處理 0 的情況

In [ ]:
for col in numerical_features:
    df[col] = np.log1p(df[col])

##數值資料標準化

In [ ]:
# 建立 scaler 物件
scaler = StandardScaler()

# 對數值特徵做標準化
df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [ ]:
# 數值欄位轉 float（浮點數）
df[numerical_features] = df[numerical_features].astype(float)

#K-means分群實作



In [ ]:
# 取出要用來分群的數值型特徵
df_clustering = df[numerical_features]

# ==========================
# 修改以下程式碼中的 n_clusters 的數值，n_clusters → 設定要分成幾群
# 說明: 透過調整分群數量，觀察是否能得到更好的分群結果
kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)

# 補充：random_state=42 → 固定隨機種子，讓結果可以重現；n_init=10 → 每次隨機選 10 組初始中心，取表現最好的結果
# ==========================

clusters = kmeans.fit_predict(df_clustering)
df['cluster'] = clusters  # 把分群結果加回原始資料

###用散佈圖顯示分群結果

In [ ]:
# PCA：把複雜的資料（很多特徵）濃縮成2個重點（二維），方便畫圖
pca = PCA(n_components=2)
X_pca = pca.fit_transform(df_clustering) # Use df_clustering here

# 把數字結果轉成表格格式
df_pca = pd.DataFrame(data=X_pca, columns=['PCA1', 'PCA2'])
df_pca['cluster'] = clusters

# 從 KMeans 物件中取得實際分的群數
K_CLUSTERS = kmeans.n_clusters

# 繪圖：用散佈圖顯示分群結果
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x='PCA1',
    y='PCA2',
    hue='cluster',
    palette=sns.color_palette("hsv", K_CLUSTERS),
    data=df_pca,
    legend="full",
    alpha=0.8
)
plt.title(f'K-Means result (k={K_CLUSTERS})', fontsize=16)
plt.xlabel('PCA1', fontsize=12)
plt.ylabel('PCA2', fontsize=12)
plt.grid(True)
plt.show()

# 判斷分群結果：群體特徵比較

表格顯示每個群的平均特徵值

* 每列 = 一個特徵（價格、評論數、房間數…）
* 每欄 = 一個群
* 例如：
  * cluster 1 的 price = 230 → 代表這群房源的平均價格是 230 元
  * cluster 2 的 bedrooms = 1.5 → 代表這群房源平均有 1.5 間房間


In [ ]:
numerical_features_to_analyze = numerical_features

# 建立一個新的資料表，只保留要分析的特徵和分群標籤
df_clustered_numerical = df[numerical_features_to_analyze + ['cluster']].copy()

# 依照群組計算每個特徵的平均值
cluster_means_scaled_logged = df_clustered_numerical.groupby('cluster')[numerical_features_to_analyze].mean()

# 把平均值還原回原始的數值（將標準化還原回去）
cluster_means_original_scale = scaler.inverse_transform(cluster_means_scaled_logged)

# 將還原後的平均值轉成表格
cluster_profile_df = pd.DataFrame(cluster_means_original_scale, columns=numerical_features_to_analyze)

# 把 log1p 還原成原始數值（np.expm1 = 反 log1p）
for col in numerical_features_to_analyze:
    cluster_profile_df[col] = np.expm1(cluster_profile_df[col])

# 把群組編號加到表格裡面
cluster_profile_df['cluster'] = cluster_means_scaled_logged.index

# 設定 cluster 為索引，方便查看每個群的特徵
cluster_profile_full = cluster_profile_df.set_index('cluster')

# 印出每個群的平均特徵值
print("Cluster Profiles:")
print(cluster_profile_full.T.round(2))